조엘 그루스, 2016, 인사이트, '밑바닥부터 시작하는 데이터 과학'

### 깁슨 샘플링 : 일부 조건부 확률을 알고 있을 때의 샘플링 방법

In [10]:
import random
import math, random, re
from collections import defaultdict, Counter

In [17]:
def roll_a_die():
    return random.choice([1,2,3,4,5,6])

#보통의 표본생성
def direct_sample():
    d1 = roll_a_die()
    d2 = roll_a_die()
    return d1, d1 + d2

#조건부 확률분포만 안다고 가정했을 때
# 두 개의 주사위 중 하나의 주사위 값(x)을 알고 그 합을 도출할 때, x를 알면 y값의 경우의 수를 알 수 있다.
def random_y_given_x(x):
    return x + roll_a_die()
#두 개의 주사위 합을 알 때 두 개의 주사위의 값의 경우의 수를 알 수 있다.
def random_x_given_y(y):
    if y<= 7:
        return random.randrange(1, y)
    else:
        return random.randrange(y - 6, 7)
random_x_given_y(11)

5

In [18]:
def gibbs_sample(num_iters =100):
    x, y = 1, 2 # x와 y를 초기화
    for _ in range(num_iters):
        x = random_x_given_y(y)
        y = random_y_given_x(x)
    return x, y
gibbs_sample()

(4, 5)

### LDA

In [133]:
# 문서별 단어리스트
# documents[3][4] => 4번째 문서의 5번째 단어
documents = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

In [119]:
# 4개의 토픽을 뽑아보자
K =4
document_topic_counts = [Counter() for _ in documents] #각 토픽이 각 문서에 할당되는 횟수
topic_word_counts = [Counter() for _ in range(K)]      #각 단어가 각 토픽에 할당되는 횟수
topic_counts = [0 for _ in range(K)]                   #각 토픽에 할당되는 총 단어 수
document_lengths = list(map(len, documents))           #각 문서에 포함되는 총 단어 수 
distinct_words = set(word for document in documents for word in document)
W = len(distinct_words) #단어 종류의 수 
D = len(documents) #총 문서의 수

#### 변수 간략 설명<br>
document_topic_counts
i번째 문서 중에서 토픽 j와 관련있는 단어의 수

>document_topic_counts[i][j]

test라는 단어가 토픽 j와 연관지어 나오는 횟수

>topic_word_counts[j]['test']

In [120]:
# d번째 문서의 총 단어 중 topic에 해당하는 단어의 비율(smoothing추가 : 0 이상의 확률을 가지도록함)
def p_topic_given_document(topic, d, alpha =0.1):
    return ((document_topic_counts[d][topic] + alpha) / (document_lengths[d] + K*alpha))
# topic에 속한 단어들 중 word의 비율(smoothing추가 : 0 이상의 확률을 가지도록함)
def p_word_given_topic(word, topic, beta=0.1):
    return ((topic_word_counts[topic][word] + beta) / (topic_counts[topic] + W*beta))

#가중치 업데이트
def topic_weight(d, word, k):
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)
#가중치를 이용해 어느정도 랜덤적으로 토픽을 반환
def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0: return i
#새로운 토픽 반환
def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

In [121]:
random.seed(0)
document_topics = [[random.randrange(K) for word in document] for document in documents]

# 임의로 할당 -> 초기화
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

In [122]:
for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d], document_topics[d])):
            #단어와 단어에 할당된 토픽을 지운다.
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1
            
            #weight를 이용해 새로운 값 도출
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            
            #새로운 값을 다시 할당한다.
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

In [123]:
document_topic_counts[:3]

[Counter({3: 0, 0: 7, 2: 0, 1: 0}),
 Counter({3: 0, 2: 0, 1: 5, 0: 0}),
 Counter({1: 2, 0: 0, 2: 2, 3: 2})]

In [124]:
#시각화

,0,1,2,3
0,Java,HBase,regression,statistics
1,Big Data,neural networks,R,probability
2,Hadoop,Postgres,libsvm,Python
3,HBase,MongoDB,scikit-learn,R
4,C++,machine learning,Python,pandas
5,Spark,Cassandra,mathematics,statsmodels
6,Storm,numpy,support vector machines,C++
7,programming languages,decision trees,Haskell,artificial intelligence
8,MapReduce,deep learning,Mahout,theory
